In [1]:
# 필요한 도구 불러오기 

import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback
from sklearn.preprocessing import LabelEncoder
from PIL import Image 
import glob
import os

In [2]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
X_train = X_train.reshape( -1, 28, 28, 1)
X_test = X_test.reshape( -1, 28, 28, 1)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5)

# 시각화를 위해 sample 데이터를 준비합니다.
X_sample = X_test[:50]
y_sample = y_test[:50]

CLASS_NAMES = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
# wandb.login(key = "c4e33984a0f1d0c7e209f455add7b4da4718e070")

In [4]:
# Sweep의 config 설정하기 

sweep_config = {
    "name": "sweep_test_core",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "learning_rate" : {
            "min" : 0.001,
            "max" : 0.1
            },
        "epoch" : {
            "distribution" : "int_uniform",
            "min" : 5,
            "max" : 10
            }
                    
        }
    }

In [5]:
# train 함수의 완결성 있는 run 구조 

def train():
    default_config = {
        "input" : (28,28,1),
        "filter" : 16,
        "kernel" : (3,3),
        "activation" : "relu",
        "learning_rate" : 0.005,
        "optimizer" : "adam",
        "loss" : "sparse_categorical_crossentropy",
        "metrics" : ["accuracy"],
        "epoch" : 5,
        "batch_size" : 32
    }

    wandb.init(config = default_config)
    config = wandb.config

    # Model

    model=keras.models.Sequential()
    model.add(keras.layers.Conv2D(config.filter, config.kernel, activation=config.activation, input_shape=config.input))
    model.add(keras.layers.MaxPool2D(2,2))
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))

    # 머신 러닝 학습때 여러가지 optimzier를 사용할 경우나 learning rate를 조절할 경우에는 아래와 같은 형태의 코드를 응용합니다.

    if config.optimizer == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate = config.learning_rate)
    
    model.compile(optimizer = optimizer,
                  loss = config.loss,
                  metrics = config.metrics)

    # WandbCallback 함수는 후술합니다.
    
    model.fit(X_train, y_train,
              epochs = config.epoch,
              batch_size = config.batch_size,
              validation_data = (X_val, y_val),
              callbacks = [WandbCallback(validation_data = (X_sample, y_sample),
                                        lables = CLASS_NAMES,
                                        predictions = 10,
                                        input_type = "images")])
    
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
    
    # wandb.log 함수 안에 기록하고 싶은 정보를 담습니다.
    
    wandb.log({"Test Accuracy Rate: " : round(test_accuracy * 100, 2),
               "Test Error Rate: " : round((1 - test_accuracy) * 100, 2)})

In [8]:
# entity와 project에 본인의 아이디와 프로젝트명을 입력하세요

sweep_id = wandb.sweep(sweep_config,
                       entity = "wisdom-jihyekim-aiffel",
                       project = "MNIST_for_Jellyfish")

# run the sweep
wandb.agent(sweep_id,
            function=train,
            count=10)

Create sweep with ID: sl1hihab
Sweep URL: https://wandb.ai/wisdom-jihyekim-aiffel/MNIST_for_Jellyfish/sweeps/sl1hihab


wandb: Agent Starting Run: khq1hc9s with config:
wandb: 	epoch: 8
wandb: 	learning_rate: 0.09886229686289316
wandb: Currently logged in as: wisdom-jihyekim (wisdom-jihyekim-aiffel). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/8
1875/1875 [==============================] - 9s 3ms/step - loss: 2.3344 - accuracy: 0.1025 - val_loss: 2.3081 - val_accuracy: 0.0974
INFO:tensorflow:Assets written to: /aiffel/aiffel/wandb/run-20241115_033939-khq1hc9s/files/model-best/assets


wandb: Adding directory to artifact (/aiffel/aiffel/wandb/run-20241115_033939-khq1hc9s/files/model-best)... Done. 0.0s


...

wandb: Agent Starting Run: p2zpr2ql with config:
wandb: 	epoch: 6
wandb: 	learning_rate: 0.0999839153097144


Epoch 1/6
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3581 - accuracy: 0.1014 - val_loss: 2.3057 - val_accuracy: 0.1026
INFO:tensorflow:Assets written to: /aiffel/aiffel/wandb/run-20241115_034831-p2zpr2ql/files/model-best/assets


wandb: Adding directory to artifact (/aiffel/aiffel/wandb/run-20241115_034831-p2zpr2ql/files/model-best)... Done. 0.0s


Epoch 2/6
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3137 - accuracy: 0.1037 - val_loss: 2.3192 - val_accuracy: 0.1046
Epoch 3/6
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3141 - accuracy: 0.1030 - val_loss: 2.3096 - val_accuracy: 0.1202
Epoch 4/6
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3135 - accuracy: 0.1038 - val_loss: 2.3237 - val_accuracy: 0.0966
Epoch 5/6
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3137 - accuracy: 0.1025 - val_loss: 2.3195 - val_accuracy: 0.1046
Epoch 6/6
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3142 - accuracy: 0.1016 - val_loss: 2.3130 - val_accuracy: 0.1202
157/157 - 0s - loss: 2.3225 - accuracy: 0.1068


Test Accuracy Rate:,▁
Test Error Rate:,▁
accuracy,▁█▆█▄▂
epoch,▁▂▄▅▇█
loss,█▁▁▁▁▁
val_accuracy,▃▃█▁▃█
val_loss,▁▆▃█▆▄
Test Accuracy Rate:,10.68
Test Error Rate:,89.32
accuracy,0.10165
best_epoch,0
